# TrustyAI
By testing our model after training and before it goes into production, we get a good understand on how well the model performs on data which we currently have available.  
Unfortunatly, the real world is not very simple and things are constantly changing, so what we took for granted yesterday may look completely different today. Because of this, we need to monitor if changes in the world has an impact on our models ability to make accurate predictions.  
We want to track key metrics that can indicate if the model or the data starts behaving strangely once the model is in production and sees live data.  
For this, we have a tool called TrustyAI that can help us generate metrics for fairness and drift detection. With TrustyAI, we can determine if live data differs from what we expected when we created our training dataset. It allows us to measure whether our model responds to real-world data differently than we initially anticipated.

### Start by installing onnxruntime
We will need this later when we produce data that will be sent to TrustyAI.

In [1]:
!pip -q install onnxruntime model-registry==0.2.10

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
codeflare-sdk 0.22.0 requires pydantic<2, but you have pydantic 2.10.3 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


### Get your User token
We need to provide our user token so that the workbench can send request to our TrustyAI Service.
You can get the user token by:
1. Going to the OpenShift Console
2. Click the dropdown in the top right corner where your username is displayed
3. Choose "Copy login command"
4. Log in
5. Select the part in the first code box after `token`, it should look something like `sha256~.....`

### Get your Model Version

We are going to fetch some artifacts from the model training pipeline we ran earlier, and we will do it by utilizing the Model Registry that keeps track of what pipeline was ran.  
To do that, we need to point out what model version we are interested in.  
Go to the Model Registry called *userX*-mlops and get the **first** model version, which is the git hash looking thing.

In [ ]:
token = "ENTER-USER-TOKEN"
model_version = "ENTER-YOUR-MODEL-VERSION"
cluster_domain = "ENTER-YOUR-CLUSTER-DOMAIN"
infer_endpoint = "ENTER-YOUR-INFERENCE-ENDPOINT"

model_name = "jukebox"

In [4]:
import pandas as pd
import pickle
import json
import onnxruntime as rt
import numpy as np
import onnx

import requests
from urllib.parse import urljoin
from fetch_artifacts_from_registry import fetch_artifacts_from_registry

### Data
We are going to send our training data to the TrustyAI Service so that it can compare the training data with the new data coming in from our inference requests.  
We limit it to just 5000 samples to keep it light, but in a real usecase you would send your full training data, or a part of it that properly represented its distribution.

In [5]:
namespace_file_path =\
    '/var/run/secrets/kubernetes.io/serviceaccount/namespace'
with open(namespace_file_path, 'r') as namespace_file:
    current_namespace = namespace_file.read()

In [6]:
artifacts = ["preprocess-data/train_data.pkl", "preprocess-data/scaler.pkl", "convert-keras-to-onnx/onnx_model.onnx", "preprocess-data/test_data.pkl"]
pipeline_namespace = f"{current_namespace}-mlops"
model_registry_url = f"https://{current_namespace}-mlops-registry-rest.{cluster_domain}"
author_name = current_namespace

In [7]:
saved_files = fetch_artifacts_from_registry(
    artifacts,
    pipeline_namespace,
    model_registry_url,
    model_name,
    model_version,
    author_name,
)

File downloaded successfully to train_data.pkl
File downloaded successfully to scaler.pkl
File downloaded successfully to onnx_model.onnx


In [8]:
X_train = pd.read_pickle(saved_files['preprocess-data/train_data.pkl'])[0][:5000]

with open(saved_files["preprocess-data/scaler.pkl"], 'rb') as handle:
    scaler = pickle.load(handle)
data = scaler.transform(X_train)

We also get the predictions of the data so that we can see if they start drifting as well.

In [9]:
sess = rt.InferenceSession(saved_files["convert-keras-to-onnx/onnx_model.onnx"], providers=rt.get_available_providers())
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
y_pred_temp = sess.run([output_name], {input_name: data.astype(np.float32).tolist()})

After we have all our data, we structure it in a specific way that TrustyAI expects.  
Noteably, we add a data_tag to our data, so that we can keep track of different iterations of it or data that's used for different purposes.

In [10]:
training_data = {
    "model_name": model_name,
    "data_tag": "TRAINING",
    "request": {
        "inputs": [
            {
                "name": input_name,
                "shape": np.shape(data),
                "datatype": "FP32",
                "data": data.tolist()
            }
        ]    
    },
    "response": {
        "model_name": model_name,
        "model_version": "1",
        "outputs": [
            {
                "name": output_name,
                "datatype": "FP32",
                "shape": np.shape(y_pred_temp[0]),
                "data": y_pred_temp[0].tolist()
            }
        ]
    }
}


### Interacting with TrustyAI through requests
We will be interacting with our TrustyAI Service through rest requests.  
Before we can do that though, we need to add our TrustyAI Service Route as the `base_url` so we know where to send the requests.

In [11]:
base_url = f"http://trustyai-service.{current_namespace}-test.svc.cluster.local"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

First thing we do is upload the data which we have prepared.

In [12]:
# Upload data
endpoint = "data/upload"
url = urljoin(base_url, endpoint)
response = requests.post(url, headers=headers, json=training_data)
print(response.text)

5000 datapoints successfully added to jukebox data.


Then we change the names of the inputs. By default they will just be the input-layer name with a number behind them, but in our case we can change them to the actual feature names.

In [13]:
# Change the input and output names
endpoint = "info/names"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "inputMapping": {
        f"{input_name}-0": "is_explicit",
        f"{input_name}-1": "duration_ms",
        f"{input_name}-2": "danceability",
        f"{input_name}-3": "energy",
        f"{input_name}-4": "key",
        f"{input_name}-5": "loudness",
        f"{input_name}-6": "mode",
        f"{input_name}-7": "speechiness",
        f"{input_name}-8": "acousticness",
        f"{input_name}-9": "instrumentalness",
        f"{input_name}-10": "liveness",
        f"{input_name}-11": "valence",
        f"{input_name}-12": "tempo"
    },
    "outputMapping": {
    }
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

Feature and output name mapping successfully applied.


Now we can subscribe to our drift detection metric which will cause our TrustyAI Service to continously publish drift (specifically meanshift) metrics.  

The meanshift metric track how different the distribution of the training data looks like compared to the new data we send it.  

For each individual input and output feature we will get a "p-value" between 0 and 1. A p-value of 1.0 indicates a very high likelihood that the train and test data come from the same distribution, while a p-value < 0.05 indicates a statistically significant drift between train and test set.


In [14]:
# Monitor meanshift
endpoint = "/metrics/drift/meanshift/request"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "referenceTag": "TRAINING"
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

{"requestId":"c3151d75-a985-4d91-a1c2-25fcacdf8a1e","timestamp":"2024-12-06T13:29:00.254+00:00"}


To make sure all looks correct, we can ask the TrustyAI Service how our current setup looks like.  
We will get back information on what metrics we have subscribed to, as well as what data has been added.  

In [15]:
# See what we have registered
endpoint = "/info"
url = urljoin(base_url, endpoint)
response = requests.get(url, headers=headers)
print(response.text)

{"jukebox":{"metrics":{"scheduledMetadata":{"metricCounts":{"MEANSHIFT":1}}},"data":{"inputSchema":{"items":{"loudness":{"type":"FLOAT","name":"input-5","columnIndex":5},"liveness":{"type":"FLOAT","name":"input-10","columnIndex":10},"tempo":{"type":"FLOAT","name":"input-12","columnIndex":12},"valence":{"type":"FLOAT","name":"input-11","columnIndex":11},"instrumentalness":{"type":"FLOAT","name":"input-9","columnIndex":9},"danceability":{"type":"FLOAT","name":"input-2","columnIndex":2},"speechiness":{"type":"FLOAT","name":"input-7","columnIndex":7},"duration_ms":{"type":"FLOAT","name":"input-1","columnIndex":1},"mode":{"type":"FLOAT","name":"input-6","columnIndex":6},"acousticness":{"type":"FLOAT","name":"input-8","columnIndex":8},"is_explicit":{"type":"FLOAT","name":"input-0","columnIndex":0},"key":{"type":"FLOAT","name":"input-4","columnIndex":4},"energy":{"type":"FLOAT","name":"input-3","columnIndex":3}},"nameMapping":{"input-0":"is_explicit","input-1":"duration_ms","input-2":"danceab

### Send a request
Finally, we need to send a single request to our model server for TrustyAI to start publishing the metrics. This is so that it has at least one inference datapoint to compare its training data to.

In [16]:
infer_url = f"{infer_endpoint}/v2/models/{model_name}/infer"

def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": input_name,
                "shape": [1, 13],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

prediction = rest_request(data[0].tolist())

/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jukebox-user14-test.apps.cluster-r44mz.r44mz.sandbox2644.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Additonally, we can also monitor the avearage value of any feature to see how it changes over time.

In [17]:
# Monitor the average value
endpoint = "/metrics/identity/request"
url = urljoin(base_url, endpoint)

payload = {
    "modelId": model_name,
    "columnName": "duration_ms",
    "batchSize": 256,
}

response = requests.post(url, headers=headers, json=payload)
print(response.text)

{"requestId":"720cb224-5e20-4259-9014-7e529d7875e2","timestamp":"2024-12-06T13:29:00.371+00:00"}


Let's go to the next Notebook to create some drift and observe the metrics in OpenShift UI 👉 [jukebox/4-metrics/2-introducing_drift.ipynb](2-introducing_drift.ipynb)